# Implementación de pipelines con scikit-learn




Este notebook es una extensión de la introducción a scikit-learn. Veremos el mismo ejemplo de clasificación pero añadiremos detalles extra



En esta sesión, aprenderemos cómo utilizar scikit-learn para construir una pipeline de Machine Learning completa. Cubriremos los siguientes pasos:

1. Recolección de Datos
2. Limpieza, Preparación y División de Datos
3. Selección, Entrenamiento, Validación y Ajuste de Hiper parámetros del modelo
4. Evaluación del Modelo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score


## 1. Recolección de Datos
Carga y Exploración de Datos

Carga de datos para el ejemplo de clasificación

In [ ]:
from sklearn.datasets import load_iris

# Cargar el conjunto de datos Iris
iris = load_iris()
X_clf = pd.DataFrame(iris.data, columns=iris.feature_names)
y_clf = pd.Series(iris.target, name='Species')

# Mostrar las primeras filas
print("Datos de Clasificación (Iris):")
display(X_clf.head())
display(y_clf.head())

Datos de Clasificación (Iris):


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


,Species
0,0
1,0
2,0
3,0
4,0


In [ ]:
print("Descripción del Conjunto de Datos:")
print(iris.DESCR)

Descripción del Conjunto de Datos:
.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distributio

## 2. Limpieza, Preparación y División de Datos

Esto incluye:


*   **Limpieza de Datos**: no vamos a hacerlo en este ejemplo
*   **Transformación de Datos**:
  *   Variables numéricas: normalización
*   **Ingeniería de Características**: no vamos a hacerlo en este ejemplo
*   **División del dataset**: haremos cross-validation en el paso 3, así que aquí solo dividiremos en training y test. Cuando hagamos cross-validation, se subdividirá el training.

### Transformación de datos

Definir Preprocesadores

In [ ]:
# Preprocesador para Clasificación
numeric_transformer_clf = Pipeline(steps=[
    ('scaler', StandardScaler())
])


### División del dataset

In [ ]:
# Dividir los datos en conjuntos de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X_clf, y_clf, test_size=0.3, random_state=0)

In [ ]:
# Verificar el tamaño de los conjuntos
print(f"Tamaño del conjunto de entrenamiento: {X_train.shape}")
print(f"Tamaño del conjunto de test: {X_test.shape}")

Tamaño del conjunto de entrenamiento: (105, 4)
Tamaño del conjunto de test: (45, 4)


## 3. Selección, Entrenamiento, Validación y Ajuste de Hiper parámetros del modelo

### Selección del Modelo (y Función de Pérdida)

Dependiendo del problema (regresión o clasificación), seleccionaremos un modelo apropiado y definiremos su función de pérdida.

Modelos para Clasificación

- Regresión Logística: Minimiza la entropía cruzada.
- K-Nearest Neighbors Classifier (KNN Classifier): Minimiza la distancia entre las predicciones y las clases reales.


### Validación Cruzada y Ajuste de Hiperparámetros

Nótese que usamos el método `GridSearchCV`: Este método hace una GridSearch para buscar los mejores hiper parámetros. Además, cada entrenamiento lo hace con cross-validation.

Por ejemplo, en regresión probamos 3 hiper parámetros y especificamos cross-validation con 5 folds, así que se hacen 15 entrenamientos (5 para cada hiperparámetro). Además, después de encontrar la mejor combinación de hiperparámetros, reentrena un modelo con el conjunto de training entero, así que finalmente se hacen 16 entrenamientos. Ese modelo es el que después se usará para hacer predicciones.

Pipeline para Clasificación

In [ ]:
# Definir el pipeline para clasificación
pipeline_clf = Pipeline(steps=[
    ('preprocessor', numeric_transformer_clf),
    ('classifier', KNeighborsClassifier())
])


In [ ]:
# Definir la cuadrícula de hiperparámetros para GridSearch
param_grid_clf = {
    'classifier': [KNeighborsClassifier()],
    'classifier__n_neighbors': [5, 7, 15]  # Solo aplicable para KNN
}



In [ ]:
# Configurar GridSearchCV
grid_search_clf = GridSearchCV(
    pipeline_clf,
    param_grid_clf,
    cv=5,
    scoring='accuracy',
    refit=True,
    n_jobs=-1)


In [ ]:
# Ajustar el modelo
grid_search_clf.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('scaler',
                                                         StandardScaler())])),
                                       ('classifier', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'classifier': [KNeighborsClassifier()],
                         'classifier__n_neighbors': [5, 7, 15]},
             scoring='accuracy')

In [ ]:
# Inspeccionar los resultados de la cross-validation
cv_results = pd.DataFrame(grid_search_clf.cv_results_)
print("Resultados de la Cross-Validation:")
print(cv_results[['param_classifier__n_neighbors', 'mean_test_score', 'std_test_score']])



Resultados de la Cross-Validation:
   param_classifier__n_neighbors  mean_test_score  std_test_score
0                              5         0.952381        0.073771
1                              7         0.933333        0.071270
2                             15         0.942857        0.055533


In [ ]:

# Mostrar los mejores parámetros
print("Mejores parámetros para Clasificación:")
print(grid_search_clf.best_params_)

Mejores parámetros para Clasificación:
{'classifier': KNeighborsClassifier(), 'classifier__n_neighbors': 5}


Nota: En este ejemplo, algunos hiperparámetros no son aplicables a ciertos modelos. Por simplicidad, elegimos solo el KNN y su hiper-parámetro `n_neighbors`

## 4. Evaluación del Modelo

Evaluación para Clasificación


In [ ]:
# Predecir en el conjunto de test
y_pred = grid_search_clf.predict(X_test)

In [ ]:
# Calcular Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy en el conjunto de test: {accuracy:.2f}")

Accuracy en el conjunto de test: 0.98


Nota: `grid_search_clf.predict()` usa un modelo entrenado con todo el conjunto de training y la mejor combinación de hiper parámetros. Se puede acceder a ese modelo con `grid_search_clf.best_estimator_`

### Inspección de las predicciones

In [ ]:
# Mostrar algunas predicciones junto con los valores reales
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results.head(10))


     Actual  Predicted
114       2          2
62        1          1
33        0          0
107       2          2
7         0          0
100       2          2
40        0          0
86        1          1
76        1          1
71        1          1
